In [2]:
import numpy as np

### gardient of LOSS w.r.t WEIGHTS
dvalues=np.array([[1.,1.,1.],[2.,2.,2.],[3.,3.,3.]])
inputs=np.array([[1.,2.,3.,2.5],
                [2.,5.,-1.,2.],
                [-1.5,2.7,3.3,-0.8]])
dweights=np.dot(inputs.T,dvalues)

print(dweights)

[[ 0.5  0.5  0.5]
 [20.1 20.1 20.1]
 [10.9 10.9 10.9]
 [ 4.1  4.1  4.1]]


In [3]:
### gardient of LOSS w.r.t BIAS
dvalues=np.array([[1.,1.,1.],[2.,2.,2.],[3.,3.,3.]])
inputs=np.array([2.0,3.0,0.5])
dbiases=np.sum(dvalues,axis=0,keepdims=True)

print(dbiases)

[[6. 6. 6.]]


In [8]:
### gardient of LOSS w.r.t INPUTS
dvalues=np.array([[1.,1.,1.],[2.,2.,2.],[3.,3.,3.]])
weights=np.array([[0.2,0.8,-0.5,1],
                [0.5,-0.91,0.26,-0.5],
                [-0.26,0.27,0.17,0.87]]).T
dweights=np.dot(dvalues,weights.T)

print(dweights)

[[ 0.44  0.16 -0.07  1.37]
 [ 0.88  0.32 -0.14  2.74]
 [ 1.32  0.48 -0.21  4.11]]


In [9]:
### Adding the backward method to DENSE LAYER CLASS
class Layer_Dense:
    ## backward pass
    def backward(self,dvalues):
        # gardient on parameter
        self.dweights=np.dot(self.inputs.T,dvalues)
        self.dbiases=np.sum(dvalues,axis=0,keepdims=True)
        self.dinputs=np.dot(dvalues,self.weights.T)


## Already defined ReLu activation class
class Activation_relu:
    def forward(self,inputs):
        self.output=np.maximum(0,inputs)

### Adding the backward method to ReLU activation class
class Activation_Relu:
    def forward(self, inputs):
        self.inputs=inputs
        self.output=np.maximum(0,inputs)
    def backward(self,dvalues):
        self.dinputs=dvalues.copy()
        self.dinputs[self.inputs<=0]=0



In [16]:
## Common loss class
class Loss:
    def calculate(self,output,y):
        sample_losses=self.forward(output,y)
        data_loss=np.mean(sample_losses)
        return data_loss



### Adding backward pass to Cross Entropy loss
class Loss_categoricalCrossEntropy(Loss):
    def backward(self, dvalues,y_true):
        samples=len(dvalues)
        labels=len(dvalues[0])
        if len(y_true.shape)==1:
            y_true=np.eye(labels)[y_true] ## one hot encoding
        self.dinputs=-y_true/dvalues
        self.dinputs=self.dinputs/samples ## normalizing


In [17]:

### Softmax activation function
class Activation_softmax:
    def forward(self,inputs):
        exp_values=np.exp(inputs-np.max(inputs,axis=1,keepdims=True))
        probablities=exp_values/np.sum(exp_values,axis=1,keepdims=True)
        self.output=probablities


## Softmax classifier: Combined softmax and Cross Entropy loss for faster backward step
class Activation_softmax_CrossEntropy_Loss:
    def __init__(self):
        self.activation=Activation_softmax()
        self.loss=Loss_categoricalCrossEntropy()
    
    def forward(self,inputs,y_true):
        self.activation.forward(inputs)
        self.outputs=self.activation.output
        return self.loss.calculate(self.outputs,y_true)
    
    def backward(self, dvalues,y_true):
        samples=len(dvalues)
        if len(y_true.shape)==2:
            y_true=np.argmax(y_true,axis=1) ## one hot encoding
        self.dinputs=dvalues.copy()
        self.dinputs[range(samples),y_true]-=1
        self.dinputs=self.dinputs/samples ## normalizing

In [18]:
softmax_outputs=np.array([[0.7,0.1,0.2],[0.1,0.5,0.4],[0.02,0.9,0.08]])
class_target=np.array([0,1,1])
softmax_loss=Activation_softmax_CrossEntropy_Loss()
softmax_loss.backward(softmax_outputs,class_target)
dvalues1=softmax_loss.dinputs
print('gradient: Combined loss and activation: ')
print(dvalues1)

gradient: Combined loss and activation: 
[[-0.1         0.03333333  0.06666667]
 [ 0.03333333 -0.16666667  0.13333333]
 [ 0.00666667 -0.03333333  0.02666667]]
